In [1]:
%cd "/content/drive/My Drive/digikala/q2"

/content/drive/My Drive/digikala/q2


In [2]:
!pip install sentence-transformers hazm 

In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import xgboost as xgb

import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer

from spacy.lang.fa import stop_words

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from hazm import word_tokenize, stopwords_list
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, recall_score, confusion_matrix, f1_score, 
                             roc_auc_score, classification_report)

from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor

from hazm import stopwords_list

from sklearn.pipeline import Pipeline
# Avoiding warnings
import warnings
import os
########### Prevent Warnings ###########
warnings.filterwarnings(action='ignore')
########### Prevent Warnings ###########

In [45]:
df_train = pd.read_csv("data/train_users.csv")
df_test = pd.read_csv("data/test_users.csv")

df_train['type'] = "train"
df_test['type'] = "test"

df = pd.concat([df_train,df_test], ignore_index=True, sort=False)

df['title_fa_product'] = df['title_fa_product'].astype('category')
df['title_fa_category'] = df['title_fa_category'].astype('category')


df['title_fa_product'] = pd.factorize( df['title_fa_product'].values )[0]
df['title_fa_category'] = pd.factorize( df['title_fa_category'].values )[0]

In [46]:
df.head()

,id,title,comment,advantages,disadvantages,title_fa_product,title_fa_category,is_buyer,verification_status,rate,type
0,0,NaN,مثل بقیه محصولات الکل دار پوست رو خشک نمیکنه,NaN,NaN,0,0,1,1.0,100.0,train
1,1,Redmi 8,با این مبلغ اگه امکانات و ارزش خرید واستون مهم...,"[""رم4\r"",""قیمت نسبت به سایر برندها\r"",""طراحی و...","[""نداشتن هندزفری""]",1,1,1,1.0,80.0,train
2,2,عالی,خوبه فقط کج و کوله بدستم رسید ولی پسرم خیلی خو...,NaN,NaN,2,2,1,1.0,100.0,train
3,3,NaN,در کل خوب بود ولی متاسفانه درب محصول شکسته بود...,NaN,NaN,3,3,1,1.0,70.0,train
4,4,NaN,من که خیلی باهاش حال کردم فقط من که همیشه L می...,NaN,NaN,4,4,1,1.0,72.0,train


In [47]:
df.fillna(" ",inplace=True)

df['title'] = df['title'].astype(str)
df['comment'] = df['comment'].astype(str)
df['advantages'] = df['advantages'].astype(str)
df['disadvantages'] = df['disadvantages'].astype(str)

In [48]:
stopwd = stopwords_list()
def preprocess(text):
    pattern = r'[\u0600-\u06FF\u0698\u067E\u0686\u06AF]+'
    tokenizer = RegexpTokenizer(pattern)
    text = tokenizer.tokenize(text)
    text = set(text)
    ttt = []
    for i in text :
        if i not in stopwd :
            ttt.append(i)
    text = ' '.join(ttt)
    return text

In [49]:
df['title'] = df['title'].apply(lambda x : preprocess(x))
df['comment'] = df['comment'].apply(lambda x : preprocess(x))
df['advantages'] = df['advantages'].apply(lambda x : preprocess(x))
df['disadvantages'] = df['disadvantages'].apply(lambda x : preprocess(x))

In [50]:
df['text'] = df['title'] + " . " + df['comment'] + " . " + df['advantages'] + " . " + df['disadvantages'] 

In [51]:
df.drop(['title','comment','advantages','disadvantages'],axis=1,inplace=True)

In [52]:
df.head()

,id,title_fa_product,title_fa_category,is_buyer,verification_status,rate,type,text
0,0,0,0,1,1,100,train,. پوست محصولات بقیه الکل خشک نمیکنه . .
1,1,1,1,1,1,80,train,. خرید نمیدی مبلغ مهمه بیخودی پول برند امکانا...
2,2,2,2,1,1,100,train,. کوله بشه پسرم لطفا بدستم خوبه توجه خوشحال ب...
3,3,3,3,1,1,70,train,. داخل محصول درب خالی مقداریش شکسته بسته . .
4,4,4,4,1,1,72,train,. گرفتم اندازه اینبار میپوشیدم ش لانگه بزرگتر...


In [53]:
X_train , y_train = df[df['type'] == 'train']['text'].values , df[df['type'] == 'train']['rate'].values
X_test = df[df['type'] == 'test']['text'].values

In [54]:
len(y_train)

264399

In [14]:
# tfidf = TfidfVectorizer(use_idf=True, max_features=20000,tokenizer=word_tokenize)

# X_total = np.concatenate((X_train,X_test))

# tfidf_train = tfidf.fit_transform(X_total)

In [57]:
RF = Pipeline([
    ('tfidf',TfidfVectorizer(use_idf=True, max_features=20000,tokenizer=word_tokenize)), 
    ('RF', RandomForestRegressor(max_depth = 10 , n_estimators=50, random_state=42))
])

In [58]:
RF.fit(X_train, y_train)
preds = RF.predict(X_test)

In [59]:
submission = pd.DataFrame({'id' : df[df['type'] == 'test']['id'].values.tolist() , 'rate':preds})

In [60]:
submission.to_csv('output_q3_11.csv',index=False)

In [22]:
haz

TypeError: ignored